In [511]:
import pandas as pd
import numpy as np
import panel as pn
import folium as fm
import param
from branca.element import Template, MacroElement
pn.extension('tabulator')
pn.extension()
pn.extension(sizing_mode="stretch_width")
import hvplot.pandas
import plotly.express as px

## (1) Define Panel Widgets

In [512]:
# Define Panel widgets
month_slider = pn.widgets.IntSlider(name='Month slider', start=1, end=12, step=1, value=4)
#month_slider

In [513]:
# Define Panel widgets
confidence_slider = pn.widgets.IntSlider(name='Confidence slider', start=50, end=100, step=1, value=75)
#confidence_slider

In [514]:
confidence_range_slider = pn.widgets.IntRangeSlider(name='Confidence',start=50, end=100, value=(50, 100), step=1)

#confidence_range_slider

In [515]:
confidence_range_slider.value[0]

50

In [516]:
month_range_slider = pn.widgets.IntRangeSlider(name='Month',start=1, end=12, value=(4, 6), step=1)

#month_range_slider

In [517]:
month_range_slider.value[1]

6

## (2) Reading Data

In [518]:
#dfm = pd.read_csv('/home/oneon/wildfire/wildfire-data/active-data/ethiopia/eth_active_fire_shape.csv')

In [519]:
#dfp = pd.read_csv('/home/oneon/wildfire/wildfire-data/eth_fire_prediction_shape.csv')

In [520]:
#dfh = pd.read_csv('/home/oneon/wildfire/wildfire-data/eth_yearly_fire.csv')

In [521]:
#dfm = pd.read_csv('https://raw.githubusercontent.com/oneon/firedataset/main/ts_active_fire_shape.csv')

In [522]:
# cache data to improve dashboard performance
if 'datamap' not in pn.state.cache.keys():

    dfm = pd.read_csv('https://raw.githubusercontent.com/oneon/firedataset/main/eth_active_fire_shape.csv')

    pn.state.cache['datamap'] = dfm.copy()

else: 

    dfm = pn.state.cache['datamap']

In [523]:
# cache data to improve dashboard performance
if 'datapre' not in pn.state.cache.keys():

    dfp = pd.read_csv('https://raw.githubusercontent.com/oneon/firedataset/main/eth_fire_prediction_shape.csv')

    pn.state.cache['datapre'] = dfp.copy()

else: 

    dfp = pn.state.cache['datapre']

In [524]:
# cache data to improve dashboard performance
if 'datahis' not in pn.state.cache.keys():

    dfh = pd.read_csv('https://raw.githubusercontent.com/oneon/firedataset/main/eth_yearly_fire.csv')

    pn.state.cache['datahis'] = dfh.copy()

else: 

    dfh = pn.state.cache['datahis']

In [525]:
dfm = dfm[dfm['REGIONNAME'].str.contains('Oromia') | dfm['REGIONNAME'].str.contains('Tigray')]

In [526]:
dfp = dfp[dfp['REGIONNAME'].str.contains('Oromia') | dfp['REGIONNAME'].str.contains('Tigray')]

In [527]:
#dfp

In [528]:
dfm.shape, dfp.shape

((7257, 13), (17024, 15))

In [529]:
# Make DataFrame Pipeline Interactive
idfm = dfm.interactive()

In [530]:
idfm.shape

In [531]:
yaxis_duration = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['24h', '48h', '7d'], 
    button_type='success'
)
yaxis_duration

map_data_pipeline = (
    idfm[
        (idfm.duration == yaxis_duration)
    ]
    .groupby(['latitude', 'longitude', 'acq_date', 'acq_time', 'REGIONNAME', 'HRname', 'duration', 'instrument', 'week']).confidence.max()
    .to_frame()
    .reset_index()  
    .reset_index(drop=True)
)

In [532]:
#map_data_pipeline.columns

In [533]:
yaxis_duration.value

'24h'

## (3) Table - Firedata 

In [534]:
active_fire_table = map_data_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 20, sizing_mode='stretch_width') 
#active_fire_table

In [535]:
#map_data_pipeline.sample(10)

In [536]:
def get_map(lat=10.7450, long=37.4897, zoom_start=7):
    return fm.Map(location=[lat,long], zoom_start=zoom_start)

map = get_map()

#pn.panel(map, height=400)

## (4) Data Active - Firedata 

In [537]:
def get_df_aqi(confidence=50):
    aqi = dfm[dfm['duration'] == yaxis_duration.value]
    aqi = aqi[aqi['confidence'] >= confidence]
    return pd.DataFrame(aqi)
    #return aqi

df_aqi = get_df_aqi()

## (5) Data Forecast - Firedata 

In [538]:
def get_dfp_aqi(confidence=50):
    aqi = dfp[dfp['month'] >= month_range_slider.value[0]] 
    aqi = aqi[aqi['month'] <= month_range_slider.value[1]]
    aqi = aqi[aqi['confidence'] >= confidence]
    return pd.DataFrame(aqi)
    #return aqi

df_aqii = get_dfp_aqi()

In [539]:
df_aqi.shape, df_aqii.shape

((878, 13), (496, 15))

In [540]:
def add_aqi_circles(map, df_aqi):
    yellow_p1  = fm.map.FeatureGroup()
    orange_p1 = fm.map.FeatureGroup()
    red_p1    = fm.map.FeatureGroup()
    
    for _, row in df_aqi.iterrows():
          
        #time = str(row.acq_time)[:-2] + ":" + str(row.acq_time)[-2:]
        html=f"""
            <h4> {row.REGIONNAME}</h4>
            <p>RegionName {row.HRname}</p>
            <ul>
                <li>Date {row.acq_date}</li>
                <li>Time {row.acq_time}</li>
                <li>Lat {row.latitude}</li>
                <li>Long {row.longitude}</li>
                <li>Confidence {row.confidence}</li>
            </ul>
            </p>
            """
        iframe = fm.IFrame(html=html, width=200, height=200)
        popup = fm.Popup(iframe, max_width=2650)
    
    
        if (row.confidence >= 50) and (row.confidence < 60):
            feature_group = yellow_p1
            customicon = 'fire-flame-yellow.png'
        elif (row.confidence >= 60) and (row.confidence < 80):
            feature_group = orange_p1
            customicon = 'fire-flame-orange.png'
        elif (row.confidence >= 80) and (row.confidence <= 100):
            feature_group = red_p1
            customicon = 'fire-flame-red.png'
        else:
            feature_group = yellow_p1
            customicon = 'fire-flame-yellow.png'
            
            
        pushpin = fm.features.CustomIcon(customicon, icon_size=(30,30))
        feature_group.add_child(
            fm.Marker(
                [row.latitude, row.longitude],
                icon=pushpin,
                popup=popup
            )
        )

    map.add_child(yellow_p1)
    map.add_child(orange_p1)
    map.add_child(red_p1)

    template = """
    {% macro html(this, kwargs) %}

    <!doctype html>
    <html lang="en">
    <head>
      <meta charset="utf-8">
      <meta name="viewport" content="width=device-width, initial-scale=1">
      <title>jQuery UI Draggable - Default functionality</title>
      <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

      <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
      <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>

      <script>
      $( function() {
        $( "#maplegend" ).draggable({
                        start: function (event, ui) {
                            $(this).css({
                                right: "auto",
                                top: "auto",
                                bottom: "auto"
                            });
                        }
                    });
    });

      </script>
    </head>
    <body>


    <div id='maplegend' class='maplegend' 
        style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
         border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>

    <div class='legend-title'>Legend Confidence</div>
    <div class='legend-scale'>
      <ul class='legend-labels'>
        <li><span style='background:yellow;opacity:0.7;'></span>50 to 60</li>
        <li><span style='background:orange;opacity:0.7;'></span>60 to 80</li>
        <li><span style='background:red;opacity:0.7;'></span>80 to 100</li>

      </ul>
    </div>
    </div>

    </body>
    </html>

    <style type='text/css'>
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 1px solid #999;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    {% endmacro %}"""

    macro = MacroElement()
    macro._template = Template(template)

    map.add_child(macro)
    
#add_aqi_circles(map, df_aqi)
#pn.panel(map, height=400)

In [541]:
def add_aqi_predict_circles(map, df_aqi):
    orange_p1  = fm.map.FeatureGroup()
    red_p1 = fm.map.FeatureGroup()
    yellow_p1 = fm.map.FeatureGroup()
    week = ""

    for _, row in df_aqi.iterrows():
        #if (row.weekname != "1"):
            #week = row.weekname
        #if (row.weekname == "1"):
            #week = row.week
    
        html=f"""
            <h4> {row.REGIONNAME}</h4>
            <p>RegionName {row.HRname}</p>
            <ul>
                <li>Month {row.month}</li>
                <li>Lat {row.latitude}</li>
                <li>Long {row.longitude}</li>
                <li>Confidence {row.confidence}</li>
            </ul>
            </p>
            """
        iframe = fm.IFrame(html=html, width=200, height=200)
        popup = fm.Popup(iframe, max_width=2650)
        
        if (row.confidence >= 50) and (row.confidence < 60):
            feature_group = yellow_p1
            customicon = 'fire-flame-yellow.png'
        elif (row.confidence >= 60) and (row.confidence < 80):
            feature_group = orange_p1
            customicon = 'fire-flame-orange.png'
        elif (row.confidence >= 80) and (row.confidence <= 100):
            feature_group = red_p1
            customicon = 'fire-flame-red.png'
        else:
            feature_group = yellow_p1
            customicon = 'fire-flame-yellow.png'
    
        pushpin = fm.features.CustomIcon(customicon, icon_size=(30,30))
        feature_group.add_child(
            fm.Marker(
                [row.latitude, row.longitude],
                icon=pushpin,
                popup=popup
            )
        )

    map.add_child(orange_p1)
    map.add_child(red_p1)
    map.add_child(yellow_p1)
    
    template = """
    {% macro html(this, kwargs) %}

    <!doctype html>
    <html lang="en">
    <head>
      <meta charset="utf-8">
      <meta name="viewport" content="width=device-width, initial-scale=1">
      <title>jQuery UI Draggable - Default functionality</title>
      <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

      <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
      <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>

      <script>
      $( function() {
        $( "#maplegend" ).draggable({
                        start: function (event, ui) {
                            $(this).css({
                                right: "auto",
                                top: "auto",
                                bottom: "auto"
                            });
                        }
                    });
    });

      </script>
    </head>
    <body>


    <div id='maplegend' class='maplegend' 
        style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
         border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>

    <div class='legend-title'>Legend Confidence</div>
    <div class='legend-scale'>
      <ul class='legend-labels'>
        <li><span style='background:yellow;opacity:0.7;'></span>50 to 60</li>
        <li><span style='background:orange;opacity:0.7;'></span>60 to 80</li>
        <li><span style='background:red;opacity:0.7;'></span>80 to 100</li>

      </ul>
    </div>
    </div>

    </body>
    </html>

    <style type='text/css'>
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 1px solid #999;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    {% endmacro %}"""

    macro = MacroElement()
    macro._template = Template(template)

    map.add_child(macro)
    
#add_aqi_predict_circles(map, df_aqii)
#pn.panel(map, height=400)

## (6) Create Map Active Fire 

In [542]:
class PanelFoliumMap(param.Parameterized):
    Confidence = param.Integer(55, bounds=(50,100))
        
    def __init__(self, **params):
        super().__init__(**params)
        self.map = get_map()
        self.folium_pane = pn.pane.plot.Folium(sizing_mode="stretch_both", min_height=500, margin=0)    
        self.view = pn.Column(
            self.param.Confidence,
            self.folium_pane,
            sizing_mode="stretch_both", height=500
        )
        self._update_map()

    @param.depends("Confidence", watch=True)
    def _update_map(self):
        self.map = get_map()
        df_aqi = get_df_aqi(confidence=self.Confidence)
        add_aqi_circles(self.map, df_aqi)
        self.folium_pane.object = self.map

        
#app = PanelFoliumMap()
#app.view

## (7) Bar chart with Forecast data

In [543]:
#dfp

In [544]:
# Make DataFrame Pipeline Interactive
idfp = get_dfp_aqi().interactive()

In [545]:
#dfp

In [546]:
#dfm = dfm[dfm['REGIONNAME'].str.contains('Oromia') | dfm['REGIONNAME'].str.contains('Tigray')]

In [547]:
# Radio buttons for District Name
yaxis_district = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Oromia', 'Tigray',],
    button_type='success'
)
month_predict_bar_pipeline = (
    idfp[
        (idfp.REGIONNAME == yaxis_district)       
    ]
    .groupby(['month']).size()
    .to_frame()
    .reset_index()
    .reset_index(drop=True)
)

In [548]:
#month_predict_bar_pipeline

In [549]:
month_predict_bar_pipeline.columns = ['month', 'REGIONNAME', 'fire_count']

In [550]:
#month_predict_bar_pipeline

In [551]:
month_predict_bar_plot = month_predict_bar_pipeline.hvplot(kind='bar', 
                                                     x='month', 
                                                     y='0', 
                                                     title='Monthly fire count forecast')
#month_predict_bar_plot

## (8) Month vs Confidence scatterplot

In [552]:
month_vs_conf_scatterplot_pipeline = (
    idfp
    .groupby(['latitude', 'longitude', 'month', 'REGIONNAME'])['confidence'].mean()
    .to_frame()
    .reset_index()
    .reset_index(drop=True)
)

In [553]:
month_vs_conf_scatterplot = month_vs_conf_scatterplot_pipeline.hvplot(x='month', 
                                                                y='confidence', 
                                                                by='REGIONNAME', 
                                                                size=80, kind="scatter", 
                                                                alpha=0.7,
                                                                legend=False, 
                                                                height=500, 
                                                                width=500)
#month_vs_conf_scatterplot

## (9) Table - Firedata Forecast 

In [554]:
month_predict_pipeline = (
    idfp[
        (idfp.month == month_slider)
    ].groupby(['latitude', 'longitude', 'month', 'REGIONNAME'])['confidence'].mean()
    .reset_index()
    .reset_index(drop=True)
)

In [555]:
#month_predict_pipeline

In [556]:
predict_table = month_predict_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 20, sizing_mode='stretch_width') 
#predict_table

## (10) Create Map Forecast 

In [557]:
class PanelFoliumMapForecast(param.Parameterized):
    Confidence = param.Integer(90, bounds=(50,100))
        
    def __init__(self, **params):
        super().__init__(**params)
        self.map = get_map()
        self.folium_pane = pn.pane.plot.Folium(sizing_mode="stretch_both", min_height=500, margin=0)    
        self.view = pn.Column(
            self.param.Confidence,
            self.folium_pane,
            sizing_mode="stretch_both", height=500
        )
        self._update_map()

    @param.depends("Confidence", watch=True)
    def _update_map(self):
        self.map = get_map()
        df_aqi = get_dfp_aqi(confidence=self.Confidence)
        add_aqi_predict_circles(self.map, df_aqi)
        self.folium_pane.object = self.map

        
#app = PanelFoliumMapForecast()
#app.view

In [558]:
checkbox = pn.widgets.Checkbox(name='MODIS', value= True, disabled = False )
checkbox

Checkbox(name='MODIS', sizing_mode='stretch_width', value=True)

In [559]:
checkbox2 = pn.widgets.Checkbox(name='VIIRS', value= True, disabled = False )
checkbox2

Checkbox(name='VIIRS', sizing_mode='stretch_width', value=True)

In [560]:
checkbox_group = pn.widgets.CheckBoxGroup(
    name='Checkbox Group', value=['Aqua', 'Tera'], options=['Aqua', 'Tera'], disabled = False,
    inline=True)

checkbox_group

CheckBoxGroup(inline=True, name='Checkbox Group', options=['Aqua', 'Tera'], sizing_mode='stretch_width', value=['Aqua', 'Tera'])

In [561]:
checkbox_group2 = pn.widgets.CheckBoxGroup(
    name='Checkbox Group', value=['S-NPP', 'NOAA 20'], options=['S-NPP', 'NOAA 20'], disabled = False,
    inline=True)

checkbox_group2

CheckBoxGroup(inline=True, name='Checkbox Group', options=['S-NPP', 'NOAA 20'], sizing_mode='stretch_width', value=['S-NPP', 'NOAA 20'])

In [562]:
fig = px.bar(dfh, x='month', y='fire_count', color='year',
       title='Wildfire Hotspot trends in Ethiopia')
#fig

In [563]:
#pn.pane.Alert('## Alert\nThis is a warning!')

## (11) Creating Dashboard

In [564]:
#Layout using Template
template = pn.template.FastListTemplate(
    site="Wildfire",
    #logo="https://panel.holoviz.org/_images/logo_stacked.png",
    title='Ethiopia Wildfire',
    sidebar=[pn.pane.Markdown("## Wildfire Prediction"), 
             pn.pane.Markdown("## Wildfire Detection"),
             pn.pane.Markdown("## Wildfire Trends"),
             #pn.pane.Markdown("#### Wild fire is one of the primary reason for climate change hazard.Track wild fire realtime and forecast wildfire"), 
             #pn.pane.PNG('fire-flame.png', sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings"),
             pn.pane.Markdown("#### Select Satellites & Instruments."),
             pn.Row(checkbox),
             pn.Row(checkbox_group),
             pn.Row(checkbox2),
             pn.Row(checkbox_group2),
             pn.pane.Markdown("#### Select Month Range"),
             pn.Row(month_range_slider)],
    main=[pn.pane.Alert('## Alert\nThere is a Fire Detection Warning!', alert_type="danger"),
          pn.pane.Markdown("# Wildfire Prediction"),
          pn.Row(pn.Column(PanelFoliumMapForecast().view),
                 pn.Column(month_slider,predict_table.panel(width=500))),
          pn.Row(pn.Column(month_vs_conf_scatterplot.panel(width=600), margin=(0,25)), 
                 pn.Column(yaxis_district, month_predict_bar_plot.panel(width=600))),
          pn.pane.Markdown("# Wildfire Detection", align="center"),
          pn.Row(pn.Column(yaxis_duration,
                           PanelFoliumMap().view), 
                 active_fire_table.panel(width=500)),
          pn.pane.Markdown("# Wildfire Trends", align="center"),
          pn.Row(pn.Column(fig))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
    main_layout="",
)
#template.show()
template.servable();